# IMPORT

In [1]:
from typing import Literal, List
import numpy as np

# FUNCTIONS

In [2]:
def is_valid_pos(i, j, n, m) -> Literal[0] | Literal[1]:
    """
    Check the coodinates are not from out of the GridWorld.

    Args:
        i (int): row index
        j (int): column index
        n (int): GridWorld row length
        m (int): GridWorld column length

    Returns:
        Literal[0] | Literal[1]: 0 means invalid (row, col) indices, 1 means valid
    """
    if (i < 0 or j < 0 or i > n - 1 or j > m - 1):
        return 0
    return 1
 
 
def get_adjacent_neighbors(val_matrix, i, j) -> List:
    """
    Function that returns all adjacent elements

    Args:
        val_matrix (np.ndarray): _description_
        i (int): row index
        j (int): column index

    Returns:
        List: List of (row, col) indices surrounding the given point, if border point then use its own indices
    """
 
    # Size of given 2d array
    n = len(val_matrix)
    m = len(val_matrix[0])
    v = []
 
    # Checking for all the possible adjacent positions
    if (is_valid_pos(i - 1, j, n, m)):      # up
        v.append((i - 1, j))
    else:
        v.append((i, j))
    
    if (is_valid_pos(i, j - 1, n, m)):      # left
        v.append((i, j - 1))
    else:
        v.append((i, j))
    
    if (is_valid_pos(i, j + 1, n, m)):      # right
        v.append((i, j + 1))
    else:
        v.append((i, j))
    
    if (is_valid_pos(i + 1, j, n, m)):      # down
        v.append((i + 1, j))
    else:
        v.append((i, j))
     
    # Returning the vector
    return v

def get_new_state_value(r, c, reward, gamma, val_matrix) -> np.ndarray:
    """
    Get the new state value based on all neighbors fetched using all the actions possible

    Args:
        r (int): row index
        c (int): column index
        reward (float): reward given to a state for performing a specific action
        gamma (float): discount
        val_matrix (np.ndarray): value array

    Returns:
        np.ndarray: new value for the state in question
    """
    state_neighbors = get_adjacent_neighbors(val_matrix, r, c)
    action_val = []
    for n in state_neighbors:
        action_val.append(reward + gamma * val_matrix[n[0]][n[1]])
    
    # get the mean as all probabilities are same
    return np.mean(action_val)

# Visualization
def log_value_matrix(val_matrix, grid) -> None:
    """
    Visualize Value matrix

    Args:
        val_matrix (np.ndarray): value array
        grid (tuple): enviornment playground, world size
    """
    res = ""
    for r in range(grid[0]):
        res += "|"
        for c in range(grid[1]):
            val = str(val_matrix[r][c])
            res += " " + val[:5].ljust(5) + " |" # format
        res += "\n"
    print(res)


def value_func(grid, val_matrix, reward, gamma=1., theta=1e-4) -> int:
    """
    Get the most optimal value of a state based on actions taken to achieve the best optimum route to reach terminal point. 

    Args:
        grid (tuple): enviornment playground, world size
        val_matrix (np.ndarray): value array
        reward (float): reward given to a state for performing a specific action
        gamma (float, optional): discount. Defaults to 1.0.
        theta (float, optional): value convergence also maximum change in the value. Defaults to 1e-4.

    Returns:
        int: Number of iteration required to reduce the error than theta
        val_matrix (np.ndarray): Updated value matrix with the best values
    """
    itr = 0
    while True:
        delta = 0
        V_new = np.copy(val_matrix)
        for r in range(grid[0]):
            for c in range(grid[1]):
                if r == c == grid[0] - 1:
                    continue
                v = val_matrix[r][c]
                V_new[r][c] = get_new_state_value(r, c, reward[r][c], gamma, val_matrix)
                delta = max(delta, abs(v - V_new[r][c]))
        val_matrix = V_new

        if itr % 10 == 0:
            print(f" Value Matrix at iteration: {itr} ".center(40, '-'))
            log_value_matrix(val_matrix, grid)

        if delta < theta:
            break
        itr +=1
    
    return itr, val_matrix

# RUN

In [3]:
gridworld = (4, 4)

V = np.zeros(gridworld)
print(f" Initial Value Array ".center(40, '-'))
print(V)
R = np.full_like(V, -1)
R[-1, -1] = 0
print(f" Reward for states ".center(40, '-'))
print(R)

iter, new_V = value_func(grid=gridworld,
                         val_matrix=V,
                         reward=R
                         )

print(f"new Value matrix: \n{new_V}\n\n Generated after {iter} iteration") 
       


--------- Initial Value Array ----------
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
---------- Reward for states -----------
[[-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]
----- Value Matrix at iteration: 0 -----
| -1.0  | -1.0  | -1.0  | -1.0  |
| -1.0  | -1.0  | -1.0  | -1.0  |
| -1.0  | -1.0  | -1.0  | -1.0  |
| -1.0  | -1.0  | -1.0  | 0.0   |

---- Value Matrix at iteration: 10 -----
| -10.8 | -10.7 | -10.5 | -10.3 |
| -10.7 | -10.5 | -10.0 | -9.41 |
| -10.5 | -10.0 | -8.74 | -6.76 |
| -10.3 | -9.41 | -6.76 | 0.0   |

---- Value Matrix at iteration: 20 -----
| -19.6 | -19.1 | -18.4 | -17.8 |
| -19.1 | -18.4 | -17.2 | -15.9 |
| -18.4 | -17.2 | -14.5 | -11.0 |
| -17.8 | -15.9 | -11.0 | 0.0   |

---- Value Matrix at iteration: 30 -----
| -26.8 | -26.1 | -24.9 | -24.0 |
| -26.1 | -25.0 | -23.1 | -21.2 |
| -24.9 | -23.1 | -19.3 | -14.4 |
| -24.0 | -21.2 | -14.4 | 0.0   |

---- Value Matrix at iteration: 40 -----
| -32.7 | -31.7 | -30.2 | -